In [898]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import math
from IPython.display import Image
import pydot

#Calculate gini index for a particular characteristic
def gini(df, column):
        gini=1
        #Get count of number of rows
        rowCount = (len(df))
        #Get list and count of unique values
        countVals = df[column].value_counts().rename_axis('unique_values').reset_index(name='counts')
        for i in range(0, (len(countVals))):
            gini=gini-(countVals.iloc[i,1]/rowCount)**2   
        return gini
    
#Determine the best possible information gain and splitting point for a characteristic
def bestSplit(df, originalGini, characteristic):
    #Get list and count of unique values
    countVals = df[characteristic].value_counts().rename_axis('unique_values').reset_index(name='counts')
    countVals = countVals.sort_values(by='unique_values') #Sort countVals by values rather than count
    #Project mean values to find candidate splitting points
    projectVals=[]
    for i in range(0, len(countVals['unique_values'])-1):
        projectVals.append((countVals['unique_values'].iloc[i]+countVals['unique_values'].iloc[i+1])/2)
    #Test data splits
    splits = pd.DataFrame(data={'projection_values': projectVals, 'information_gain': np.nan})
    #For each of the possible splitting points
    for i in range(0, len(projectVals)):
        #Load dataframe into left and right nodes    
        leftData=df.copy()
        rightData=df.copy()
        for j in range(0, len(df)): #For the length of the dataframe
            if (rightData[characteristic].iloc[j] < projectVals[i]): #For any values less than projectVals[i]
                rightData[characteristic].iloc[j] = np.nan #Set row in right side as NaN
            else: #Otherwise, values are greater than or equal to projected value
                leftData[characteristic].iloc[j] = np.nan #Set row in left side as NaN
        #Delete rows with nan values for both left and right side
        leftData=leftData.dropna()
        rightData=rightData.dropna()
        #Calculate gini values for left and right nodes
        leftGini=gini(leftData, 'species')
        rightGini=gini(rightData, 'species')
        #Calculate information gain and append to splits df
        combinedGini=((len(leftData)/len(df))*leftGini)+((len(rightData)/len(df))*rightGini)
        informationGain=originalGini-combinedGini
        splits['information_gain'].iloc[i]=informationGain
    #Locate the best split point
    splitPoint=splits['projection_values'].iloc[splits['information_gain'].idxmax()]
    maxGain = splits['information_gain'].value_counts().idxmax()
    return maxGain, splitPoint

#Find best information gain over all of the characteristics and then split the data accordingly
def split(df):
    #Calculate original gini
    originalGini = gini(df, 'species')

    #Get characteristic names
    columnNames=list(df.columns.values)
    columnNames.remove(columnNames[len(columnNames)-1])

    #Determine which is best to perform split
    charSplit = pd.DataFrame(data={'characteristic': columnNames, 'information_gain': np.nan, 'splitting_point': np.nan})
    for i in range (0, len(columnNames)):
        charInformationGain, charSplitPoint = bestSplit(df, originalGini, columnNames[i])
        charSplit['information_gain'].iloc[i]=charInformationGain
        charSplit['splitting_point'].iloc[i]=charSplitPoint
        #print('Characteristic: ', columnNames[i], 'Max Information Gain:', charInformationGain,'Splitting Point:',charSplitPoint)
    splitChar=charSplit['characteristic'].iloc[charSplit['information_gain'].idxmax()]
    splitPoint=charSplit['splitting_point'].iloc[charSplit['information_gain'].idxmax()]

    #Actually split the data
    #Load dataframe into left and right nodes
    leftData = df.copy()
    rightData = df.copy()
    for i in range(0, len(df)): #For the length of the dataframe
        if (rightData[splitChar].iloc[i] < splitPoint): #For any values less than projectVals[i]
            rightData[splitChar].iloc[i] = np.nan #Set row in right side as NaN
        else: #Otherwise, values are greater than or equal to projected value
            leftData[splitChar].iloc[i] = np.nan #Set row in left side as NaN
    #Delete rows with nan values for both left and right side
    leftData=leftData.dropna()
    rightData=rightData.dropna()
    return splitChar, splitPoint, leftData, rightData

#Define a descision tree object
class DecisionTree:
    def __init__(self, df, identifier):
        #Define graph
        self.graph = pydot.Dot(graph_type='graph')
        #Define internal node variables
        self.data=df
        self.species=df[identifier].value_counts().idxmax()
        self.gini=gini(df, identifier)
        if ((self.gini>0.02) and (len(self.data)>5)):
            #print('Splitting with gini: ',self.gini,' and length: ', len(self.data))
            self.splitChar, self.splitPoint, self.leftChildData, self.rightChildData = split(df) 
            self.leftChild = DecisionTree(self.leftChildData, identifier)               
            self.rightChild = DecisionTree(self.rightChildData, identifier)
        else:
            #print('Done Splitting')
            self.leftChild = None
            self.rightChild = None
        #Define nodal information
        self.nodeInformation='Major Species: '+str(self.species)
        self.nodeInformation=self.nodeInformation+'\nNumber of Members: '+str(len(self.data))
        self.nodeInformation=self.nodeInformation+'\nGini: '+str(self.gini)
        if (self.leftChild is not None):
            self.nodeInformation = self.nodeInformation+'\n(Left): '+self.splitChar+'<'+str(self.splitPoint)
            self.leftEdge = pydot.Edge(self.nodeInformation, self.leftChild.nodeInformation)
            self.graph.add_edge(self.leftEdge)
            self.rightEdge = pydot.Edge(self.nodeInformation, self.rightChild.nodeInformation)
            self.graph.add_edge(self.rightEdge)

#Build the full tree from each sub-tree found for each node within a decision tree object
def buildGraph(tree):
    finalGraph = pydot.Dot(graph_type='graph') #Create a blank tree to hold all sub-trees
    root = tree.graph #Establish the tree's root sub-tree
    for i in range(0,len(root.get_edges())):
        finalGraph.add_edge(root.get_edges()[i])
    if (tree.leftChild is not None): #If there is a further sub-tree
        a = buildGraph(tree.leftChild) #Recursive call for left hand child 
        for i in range(0,len(a.get_edges())): #For all of the left hand child's edges
            finalGraph.add_edge(a.get_edges()[i])  #Add them to the final graph
        b = buildGraph(tree.rightChild) #Recursive call for right hand child 
        for i in range(0,len(b.get_edges())): #For all of the right hand child's edges
            finalGraph.add_edge(b.get_edges()[i]) #Add them to the final tree
    return finalGraph #Return back up the final tree

#Determine what the tree says the species should be 
def determine(startNode, dataPoint, identifier):
    if (startNode.leftChild is not None):
        if (dataPoint[startNode.splitChar]<tree.splitPoint):
            startNode = startNode.leftChild
        else:
            startNode =  startNode.rightChild
        determine(startNode, dataPoint, identifier)
        return(startNode.species)
    
#Test if a correct answer is obtained through the decision tree for a sample
def test(tree, testData, identifier):
    successes=0
    for i in range (0,len(testData)): #For each of the test data cases
        if (testData.iloc[i][identifier] == determine(tree, testData.iloc[i], 'species')):
            successes=successes+1
    return successes

#For each characteristic, find the greatest information gain possible, then split the data adding that node to a tree
#Repeat until termination criteria and then print out the tree to tree.png
print('ALLOCATING TRAINING/TESTING SETS')
trainSplit = 0.5 #Indicate portion of data to use for training; test is 1-trainSplit
filename = 'iris.csv' #Indicate filename containing dataset
iris = pd.read_csv(filename, na_values='?') #Read in dataset
iris = shuffle(iris) #Randomize iris dataset
numTrain = int(len(iris.index)*trainSplit) #Find number of training examples
trainingData =iris.iloc[0:numTrain] #Split off a training dataset
testData = iris.iloc[numTrain:(len(iris.index))] #Split off a test dataset

print('BUILDING TREE (THIS WILL TAKE A WHILE)')
tree = DecisionTree(trainingData,'species') #Construct a decision tree for determining class
finalGraph = buildGraph(tree) #Reconstruct full tree from descision tree object's sub-trees
finalGraph.write_png('tree.png') #Write the full tree to a file
Image(filename='tree.png') 

print('EVALUATING SUCCESS OF TREE FOR TESTING SET')
successes = test(tree, testData, 'species')
print('Test Samples:',len(testData),'Successes:',successes,'Failures:',len(testData)-successes)
print('Success Rate for Test Data:',(successes/len(testData))*100)